In [1]:
from Crypto.Cipher import AES
import scrypt, os, binascii
import csv
import itertools

In [169]:
def generate_key(passwd, N=16384, r=8, p=1, sklen=32, saltlen=16):
    """
    * passwd:              str or str encoded as bype object
    * saltlen:             8 bytes minimum, 16 bytes recommended
    * salt:                saltlen-byte (non-secret) random data for scrypt key derivation function
    * secretKey:           bytes object of length sklen
        * N:               iterations count (a power of 2 greater than 1) (e.g. 16384, 2048)
        * r:               block size 
        * p:               parallelism factor (where r*p < 2^30)
        * sklen:           no. of bytes to generate as output (<= (2**32 - 1)*32)
    * Memory required = 128 * N * r * p bytes
    """
    if type(passwd) == str:
        passwd = passwd.encode()
    salt = os.urandom(saltlen)
    secretKey = scrypt.hash(passwd, salt, N, r, p, sklen)
    return secretKey, salt
    
def encrypt_AES_GCM(msg, secretKey, salt):
    '''
    * msg:                 str or str encoded as bype object
    * aesCipher:           AES-256 cipher in Galois/Counter Mode
    * aesCipher.nonce:     nonce bype object
    * ciphertext, authTag: ciphertext and MAC tag
    '''
    if type(msg) == str:
        msg = msg.encode()
    aesCipher = AES.new(secretKey, AES.MODE_GCM)
    ciphertext, authTag = aesCipher.encrypt_and_digest(msg)
    return (salt, ciphertext, aesCipher.nonce, authTag)

def decrypt_AES_GCM(encryptedMsg, secretKey):
    """
    * encryptedMsg:        quadruple (salt, ciphertext, nonce, authTag)
    * plaintext:           plaintext as byte object
    """
    (salt, ciphertext, nonce, authTag) = encryptedMsg
    aesCipher = AES.new(secretKey, AES.MODE_GCM, nonce)
    plaintext = aesCipher.decrypt_and_verify(ciphertext, authTag)
    return plaintext

In [197]:
def encrypt_CSV(file_name, KEK):
    file = open(file_name, "r")
    reader_, reader = itertools.tee(csv.reader(file, delimiter=','))
    columns = len(next(reader_))
    del reader_
    
    DEKs = [] # one per column
    for _ in range(columns):
        DEKs += [generate_key(KEK)]
    
    encrypted_data = []
    for line in reader:
        encrypted_row = []
        for j in range(len(line)):
            if line[j]:
                encrypted_row.append(encrypt_AES_GCM(line[j], DEKs[j][0], DEKs[j][1]))
            else:
                encrypted_row += [[]]
        encrypted_data.append(encrypted_row)
                
    # @TODO: print to new CSV
    print(encrypted_data)
    file.close()    

* * *

# Test area

In [199]:
msg = "The Hitchhiker's Guide to the Galaxy has a few things to say on the subject of towels. A towel, it says, is about the most massively useful thing an interstellar hitch hiker can have. Partly it has great practical value—you can wrap it around you for warmth as you bound across the cold moons of Jaglan Beta; you can lie on it on the brilliant marble‐sanded beaches of Santraginus V, inhaling the heady sea vapours; you can sleep under it beneath the stars which shine so redly on the desert world of Kakrafoon; use it to sail a mini raft down the slow heavy river Moth; wet it for use in hand‐to‐hand‐combat; wrap it round your head to ward off noxious fumes or to avoid the gaze of the Ravenous Bugblatter Beast of Traal (a mindbogglingly stupid animal, it assumes that if you can't see it, it can't see you—daft as a brush, but very ravenous); you can wave your towel in emergencies as a distress signal, and of course dry yourself off with it if it still seems to be clean enough."
#msg = ['All`s Well That Ends Well', 'As You Like It', 'Comedy of Errors', 'Love`s Labour`s Lost', 'Measure for Measure', 'Merchant of Venice', 'Merry Wives of Windsor', 'Midsummer Night`s Dream', 'Much Ado about Nothing', 'Taming of the Shrew', 'Tempest', 'Twelfth Night', 'Two Gentlemen of Verona', 'Winter`s Tale']
password = "myP@ssw0rd!"

DEK, salt = generate_key(password)
encryptedMsg = encrypt_AES_GCM(msg, DEK, salt)
print("encryptedMsg", {
    'kdfSalt': binascii.hexlify(encryptedMsg[0]),
    'ciphertext': binascii.hexlify(encryptedMsg[1]),
    'aesIV': binascii.hexlify(encryptedMsg[2]),
    'authTag': binascii.hexlify(encryptedMsg[3])
})

encryptedMsg {'kdfSalt': b'5e18fafbe8231bac294bf91a0acfc309', 'ciphertext': b'8aac055c2f8b6b4cbff0e08025aecb1014dfa87f062f6c7dfbd62d705afff2df1acffa96b668003b10c2ec6dca192fe0cb0b618ac5d9d498db6a4165418f5be7cf3ece1b6ae0fe9530463b1b6cd6e9db164d3319fad54e3ecfd813cc198b632afdfce5354fb8c9da66f1128b9d6452e30a045c5c59557901147dac2d8cc81d57d0acfdddcc8c77ea67f71d4947fea6f4898c821bf460f124af14b1b9861c1d634557cd1752e835e7fb2cd4734bfd39ce2537678bb85a5ca3da2403b4dccf181bc5cbe974e38adc1fa57aa6ce2e8044bcc9a567f1cf9787b002be2107eb3194f280061819d18d30eef1d8ed14e3fbfe9cedc235f4ca5589710d0abb7ba1fda011655e8c71491f42870956aec82015b4fe7e5a107b76531570b8c014421c35c38ee3e72613f5e818d5b08a7c764b2a1cce9581e818d58038502d141819b06152e5665a3ec77f3c69da307d97e8a3477d7c1eba07d91076fa0b168ea5bf8375826944b36ab86fc83bc1c79fcdcd6ad4f20250cc1fcd65ada9e6bf54dbea12d4b4c0d2380d36381102d9c1bb16c7b9bafc481a5341361cfb8fd64abd134ce5f31ca0734c7c6de3b880ce46188d9ad5db1d613cfefc448d57517945ccd4489de873d9302e824402e862097d438ee480

In [104]:
decryptedMsg = decrypt_AES_GCM(encryptedMsg, DEK)
print("Decrypted message:", decryptedMsg.decode())

Decrypted message: The Hitchhiker's Guide to the Galaxy has a few things to say on the subject of towels. A towel, it says, is about the most massively useful thing an interstellar hitch hiker can have. Partly it has great practical value—you can wrap it around you for warmth as you bound across the cold moons of Jaglan Beta; you can lie on it on the brilliant marble‐sanded beaches of Santraginus V, inhaling the heady sea vapours; you can sleep under it beneath the stars which shine so redly on the desert world of Kakrafoon; use it to sail a mini raft down the slow heavy river Moth; wet it for use in hand‐to‐hand‐combat; wrap it round your head to ward off noxious fumes or to avoid the gaze of the Ravenous Bugblatter Beast of Traal (a mindbogglingly stupid animal, it assumes that if you can't see it, it can't see you—daft as a brush, but very ravenous); you can wave your towel in emergencies as a distress signal, and of course dry yourself off with it if it still seems to be clean enou

***

In [200]:
def encrypt_file(file_name, secretKey, salt):
    with open(file_name, 'rb') as file:
        plaintext = file.read()
    (salt, ciphertext, nonce, authTag) = encrypt_AES_GCM(plaintext, secretKey, salt)
    # with open(file_name + ".enc", 'wb') as f:
    #     f.write(salt + b'\n')
    #     f.write(ciphertext + b'\n')
    #     f.write(nonce + b'\n')
    #     f.write(authTag + b'\n')  
    # this can be done more efficiently
    with open(file_name + ".salt.txt", 'wb') as f:
        f.write(salt)
    with open(file_name + ".enc", 'wb') as f:
        f.write(ciphertext)
    with open(file_name + ".nonce.txt", 'wb') as f:
        f.write(nonce)
    with open(file_name + ".authTag.txt", 'wb') as f:
        f.write(authTag)  
            
def decrypt_file(file_name, secretKey):
    with open(file_name[:-4] + ".salt.txt", 'rb') as f:
        salt = f.read()
    with open(file_name) as f:
        ciphertext = f.read()
    with open(file_name[:-4] + ".nonce.txt", 'rb') as f:
        nonce = f.read()
    with open(file_name[:-4] + ".authTag.txt", 'rb') as f:
        authTag = f.read()
    (salt, ciphertext, nonce, authTag) = encryptedMsg
    plaintext = decrypt_AES_GCM(encryptedMsg, secretKey)
    with open(file_name[:-4], 'wb') as f:
        f.write(plaintext)

In [201]:
password = "myP@ssw0rd!"
DEK, salt = generate_key(password)
encrypt_file('/Users/dzackon/Desktop/testfile.txt', DEK, salt)

In [ ]:
decrypt_AES_GCM('/Users/dzackon/Desktop/testfile.txt.enc', DEK)